In [24]:
import numpy as np
from math import log2
import pandas as pd

In [25]:
def Gini(y):
    classes, counts = np.unique(y, return_counts=True)
    impurity = 1.0
    for count in counts:
        prob = count / len(y)
        impurity -= prob ** 2
    return impurity


def Entropy(y):
    classes, counts = np.unique(y, return_counts=True)
    entropy = 0.0
    for count in counts:
        prob = count / len(y)
        entropy -= prob * np.log2(prob)
    return entropy

In [32]:
class Node:
    def __init__(self, criterionValue, samples, value, featureIndex=None, threshold=None, left=None, right=None):
        self.criterionValue = criterionValue
        self.samples = samples
        self.value = value
        self.featureIndex = featureIndex
        self.threshold = threshold
        self.left = left
        self.right = right

In [26]:
class DecisionTreeClassifier:
    def __init__(self, criterion="gini", maxDepth=None, minSamplesSplit=2):
        self.criterion = criterion
        self.maxDepth = maxDepth
        self.minSamplesSplit = minSamplesSplit
        self.tree = None

    def CriterionFunction(self, y):
        if self.criterion == "gini":
            return Gini(y)
        elif self.criterion == "entropy":
            return Entropy(y)
        else:
            raise ValueError("Unknown criterion: '{}'".format(self.criterion))

    def SplitDataset(self, X, y, featureIndex, threshold):
        leftMask = X[:, featureIndex] <= threshold
        rightMask = X[:, featureIndex] > threshold
        return X[leftMask], X[rightMask], y[leftMask], y[rightMask]

    def BestSplit(self, X, y):
        bestCriterionValue = float('inf')
        bestIndex = None
        bestThreshold = None
        
        for featureIndex in range(X.shape[1]):
            thresholds = np.unique(X[:, featureIndex])
            for threshold in thresholds:
                X_left, X_right, y_left, y_right = self.SplitDataset(X, y, featureIndex, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                
                criterionLeft = self.CriterionFunction(y_left)
                criterionRight = self.CriterionFunction(y_right)
                criterionSplit = (len(y_left) * criterionLeft + len(y_right) * criterionRight) / len(y)
                
                if criterionSplit < bestCriterionValue:
                    bestCriterionValue = criterionSplit
                    bestIndex = featureIndex
                    bestThreshold = threshold
                    
        return bestIndex, bestThreshold

    def BuildTree(self, X, y, depth=0):
        numSamples = len(y)
        numClasses = len(np.unique(y))
        
        if depth == self.maxDepth or numSamples < self.minSamplesSplit or numClasses == 1:
            leafValue = np.bincount(y).argmax()
            return Node(criterionValue=self.CriterionFunction(y), samples=numSamples, value=leafValue)
        
        featureIndex, threshold = self.BestSplit(X, y)
        if featureIndex is None:
            leafValue = np.bincount(y).argmax()
            return Node(criterionValue=self.CriterionFunction(y), samples=numSamples, value=leafValue)
        
        X_left, X_right, y_left, y_right = self.SplitDataset(X, y, featureIndex, threshold)
        leftNode = self.BuildTree(X_left, y_left, depth + 1)
        rightNode = self.BuildTree(X_right, y_right, depth + 1)
        return Node(criterionValue=self.CriterionFunction(y), samples=numSamples, value=None, 
                    featureIndex=featureIndex, threshold=threshold, 
                    left=leftNode, right=rightNode)

    def Fit(self, X, y):
        self.tree = self.BuildTree(X, y)

    def PredictTree(self, node, x):
        if node.value is not None:
            return node.value
        if x[node.featureIndex] <= node.threshold:
            return self.PredictTree(node.left, x)
        else:
            return self.PredictTree(node.right, x)

    def Predict(self, X):
        return [self.PredictTree(self.tree, x) for x in X]

    def PrintTree(self, node=None, depth=0):
        if node is None:
            node = self.tree
        
        if node.value is not None:
            print(f"{'|  ' * depth}-- Leaf: Class={node.value} (Samples={node.samples}, {self.criterion.capitalize()}={node.criterionValue:.3f})")
        else:
            print(f"{'|  ' * depth}-- Node: FeatureIndex={node.featureIndex}, Threshold={node.threshold:.3f} (Samples={node.samples}, {self.criterion.capitalize()}={node.criterionValue:.3f})")
            print(f"{'|  ' * (depth + 1)}Left:")
            self.PrintTree(node.left, depth + 1)
            print(f"{'|  ' * (depth + 1)}Right:")
            self.PrintTree(node.right, depth + 1)

In [28]:
df = pd.read_csv("./Data/exams.csv")
df.head()

,exam_1,exam_2,admitted
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [29]:
X = df.drop("admitted", axis = 1).to_numpy()
y = df.admitted.to_numpy()

In [30]:
X.shape
y.shape

(100,)

In [31]:
classifier = DecisionTreeClassifier(criterion="entropy", maxDepth=3)
classifier.Fit(X, y)

# Make predictions
predictions = classifier.Predict(X)
print("Predictions:", predictions)

# Print the tree structure
print("\nDecision Tree Structure:")
classifier.PrintTree()

Predictions: [0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

Decision Tree Structure:
-- Node: FeatureIndex=0, Threshold=56.254 (Samples=100, Entropy=0.971)
|  Left:
|  -- Node: FeatureIndex=1, Threshold=63.128 (Samples=35, Entropy=0.722)
|  |  Left:
|  |  -- Leaf: Class=0 (Samples=18, Entropy=0.000)
|  |  Right:
|  |  -- Node: FeatureIndex=0, Threshold=40.237 (Samples=17, Entropy=0.977)
|  |  |  Left:
|  |  |  -- Leaf: Class=0 (Samples=8, Entropy=0.000)
|  |  |  Right:
|  |  |  -- Leaf: Class=1 (Samples=9, Entropy=0.764)
|  Right:
|  -- Node: FeatureIndex=1, Threshold=42.838 (Samples=65, Entropy=0.690)
|  |  Left:
|  |  -- Leaf: Class=0 (Samples=9, Entropy=0.000)
|  |  Right:
|  |  -- Node: FeatureIndex=1, Threshold=52.061 (Samples=56,